# Interval scores

The choice of interval score represents the choice of distributional feature(s) to detect changes in.

Interval scores are not primarly meant to be used directly, but they are important building blocks to understand to make full use of the library.